## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF2 PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [68]:
reader = PdfReader("me/profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [69]:
reader = PdfReader("me/projects.pdf")
projects = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        projects += text

In [70]:
print(linkedin)
print(projects)

   
Contact
yosola@gmail.com
www.linkedin.com/in/crissrodriguez
(LinkedIn)
github.com/Yosolita1978
(Personal)
yosola.co/ (Portfolio)
Top Skills
JSON
Cascading Style Sheets (CSS)
Stable Diffusion
Languages
English
Certifications
AI Agents Fundamentals
Badge for Introduction to Generative
AI Introduction to Generative AI
Android Basics Nanodegree program
Cristina Rodríguez
Technical Curriculum Developer | Software Engineer Instructor |
Women Techmakers Ambassador | Passionate about #womenintech
Greater Seattle Area
Summary
As a Technical Curriculum Developer at ELM Learning, I specialize
in designing impactful, learner-centered training programs that
simplify complex technical concepts for diverse audiences. I bring
over 20 years of experience in project management, content
development, and communications, paired with a deep passion for
inclusive tech education.
Previously at Techtonica, I led cross-functional initiatives as
a Technical Project Manager, spearheading event planning,
volun

In [71]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [72]:
name = "Cristina Rodriguez"

In [73]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n\n\n## Projects:\n{projects}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [74]:
system_prompt

"You are acting as Cristina Rodriguez. You are answering questions on Cristina Rodriguez's website, particularly questions related to Cristina Rodriguez's career, background, skills and experience. Your responsibility is to represent Cristina Rodriguez for interactions on the website as faithfully as possible. You are given a summary of Cristina Rodriguez's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Cristina Rodriguez. I'm a software engineer, technical curriculum developer, and learning experience designer. I'm originally from Colombia but now live in Seattle, WA. I’m passionate about educational technology and making complex technical topics easy to grasp. I specialize in front-end development with React and mobile technologies, but I also enjoy diving into backend tools like Node

In [75]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [76]:
gr.ChatInterface(chat, type="messages").launch(share=True)

* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://9b4af976432880e44b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [25]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [26]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [27]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [58]:
from dotenv import load_dotenv
import os
from google import genai

load_dotenv()

client = genai.Client(api_key=os.environ.get("GOOGLE_API_KEY"))

try:
    # Use the correct method for genai.Client
    test_response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents="Hello"
    )
    print("✅ API key is working!")
    print(f"Response: {test_response.text}")
except Exception as e:
    print(f"❌ API key test failed: {e}")

    


✅ API key is working!
Response: Hi there! How can I help you today?



In [59]:
gemini = OpenAI(
    api_key=os.environ.get("GOOGLE_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [60]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [61]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [62]:
reply

"No, I do not currently hold a patent. My focus has primarily been on software engineering, technical curriculum development, and creating accessible learning experiences in educational technology. However, I'm always exploring new tools and technologies, and who knows what the future holds! If you have any related questions or need assistance, feel free to ask."

In [63]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="This is a great answer. It's very professional, personable and true to the provided information.")

In [64]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [65]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [67]:
gr.ChatInterface(chat, type="messages").launch(share=True)

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://becb601faa2f013d6a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Passed evaluation - returning reply
Failed evaluation - retrying
This response is not acceptable. The agent responded in pig latin, which is not professional, engaging, or helpful for the user. The response should be redone to respond accurately and professionally about whether Cristina has any patents.
